In [1]:
import os
import sys
from os.path import join
import json

from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import nfl_data_py as nfl

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, os.path.join(ROOT_DIR,'py'))

import util
from plot.plot_simple import plot_play_with_speed

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

with open("paths.json", 'r') as f:
    paths = json.load(f)

PROCESSED_DATA_PATH = paths['processed_data']

In [10]:
WEEK = 1

df_playdf_tracking = pd.read_pickle(join(PROCESSED_DATA_PATH, f'wk{WEEK}', 'tracking_final.pkl'))
df_game = pd.read_pickle(join(PROCESSED_DATA_PATH, f'wk{WEEK}', 'games_final.pkl'))
df_play = pd.read_pickle(join(PROCESSED_DATA_PATH, f'wk{WEEK}', 'play_final.pkl'))
df_player_play = pd.read_pickle(join(PROCESSED_DATA_PATH, f'wk{WEEK}', 'player_play_final.pkl'))
df_player = pd.read_pickle(join(PROCESSED_DATA_PATH, 'players.pkl'))
df_team = pd.read_pickle(join(PROCESSED_DATA_PATH, 'teams.pkl'))

# Drop plays where QB is the ball carrier

In [3]:
(
    df_player_play
    [['game_play_id','had_rush_attempt','nfl_id']]
    .query('had_rush_attempt == 1')
    .merge(df_player[['nfl_id','position']], on='nfl_id', how='left')
    .position.value_counts()
)

RB    552
WR     19
FB      4
QB      2
Name: position, dtype: int64

In [23]:
gpids_with_qb_rusher = (
    df_player_play
    [['game_play_id','had_rush_attempt','nfl_id']]
    .query('had_rush_attempt == 1')
    .merge(df_player[['nfl_id','position']], on='nfl_id', how='left')
    .query('position == "QB"')
    .game_play_id
    .unique()
    .tolist()
)
game_ids_with_qb_rusher = df_play.query('game_play_id.isin(@gpids_with_qb_rusher)').game_id.unique().tolist()

if len(game_ids_with_qb_rusher) != 0:
    df_tracking = df_tracking.query('~game_play_id.isin(@gpids_with_qb_rusher)')
    df_play = df_play.query('~game_play_id.isin(@gpids_with_qb_rusher)')
    df_player_play = df_player_play.query('~game_play_id.isin(@gpids_with_qb_rusher)')
    df_game = df_game.query('~game_id.isin(@game_ids_with_qb_rusher)')
    df_


['2022091804_222', '2022091804_720']

# Verify all plays have 5 labeled oline players (LT, LT, C, RT, RT)

In [22]:
all_gpids = set(df_tracking.game_play_id.unique().tolist())

gpids_with_oline = (
    set(
        df_tracking
        .query('position_by_loc.isin(["LT","LG","C","RG","RT"])')
        .game_play_id
        .unique()
        .tolist()
    )
)

all_gpids - gpids_with_oline

set()

In [9]:
cols = [
    'game_play_id',
    'rush_location_type','run_location_desc','run_location',
    'pff_run_concept_primary','pff_run_concept_secondary',
    'offense_personnel','offense_formation','receiver_alignment','play_action','pff_run_pass_option',
    'defense_personnel','defenders_in_box'
]
df_play[cols].head(40)

,game_play_id,rush_location_type,run_location_desc,run_location,pff_run_concept_primary,pff_run_concept_secondary,offense_personnel,offense_formation,receiver_alignment,play_action,pff_run_pass_option,defense_personnel,defenders_in_box
0,2022091806_346,OUTSIDE_RIGHT,middle,middle,MAN,NaN,"1 RB, 1 TE, 3 WR",SHOTGUN,2x2,False,0,"3 DL, 3 LB, 5 DB",6.0
1,2022091804_699,INSIDE_RIGHT,left_tackle,left,OUTSIDE ZONE,NaN,"1 RB, 1 TE, 3 WR",SINGLEBACK,2x2,False,0,"2 DL, 4 LB, 5 DB",7.0
2,2022091811_120,INSIDE_RIGHT,middle,middle,TRAP,LEAD,"2 RB, 1 TE, 2 WR",I_FORM,2x1,False,0,"3 DL, 4 LB, 4 DB",7.0
3,2022091806_950,INSIDE_RIGHT,right_tackle,right,OUTSIDE ZONE,NaN,"1 RB, 3 TE, 1 WR",SINGLEBACK,2x2,False,0,"3 DL, 3 LB, 5 DB",8.0
4,2022091810_3528,OUTSIDE_RIGHT,middle,middle,MAN,NaN,"1 RB, 1 TE, 3 WR",SINGLEBACK,2x2,False,0,"4 DL, 2 LB, 5 DB",6.0
5,2022091804_2966,INSIDE_LEFT,left_guard,left,MAN,NaN,"1 RB, 2 TE, 2 WR",SINGLEBACK,3x1,False,0,"5 DL, 2 LB, 4 DB",8.0
6,2022091807_1196,OUTSIDE_RIGHT,right_tackle,right,OUTSIDE ZONE,NaN,"2 RB, 2 TE, 1 WR",SINGLEBACK,2x2,False,0,"3 DL, 4 LB, 4 DB",8.0
7,2022091804_4294,OUTSIDE_RIGHT,left_guard,left,OUTSIDE ZONE,NaN,"1 RB, 2 TE, 2 WR",SINGLEBACK,3x1,False,0,"4 DL, 3 LB, 4 DB",7.0
8,2022091811_141,INSIDE_LEFT,left_guard,left,COUNTER,CROSS LEAD,"2 RB, 1 TE, 2 WR",SHOTGUN,2x1,False,0,"3 DL, 4 LB, 4 DB",7.0
9,2022091809_122,INSIDE_RIGHT,left_tackle,left,OUTSIDE ZONE,NaN,"1 RB, 2 TE, 2 WR",SINGLEBACK,3x1,False,0,"5 DL, 2 LB, 4 DB",7.0


In [16]:
# Outsize Zone
gpid = "2022091812_3298"
plot_play_with_speed(
    df_tracking.query('ball_snap_fid <= frame_id <= ball_snap_fid + 10 and game_play_id==@gpid'),
    gpid, 
    every_other_frame=True, 
    event_col='event_new',
    plot_motion=False,
    show_motion_frames=False,
    highlight_offensive_positions=True,
)

In [18]:
gpid = "2022091803_795"
plot_play_with_speed(
    df_tracking.query('ball_snap_fid <= frame_id <= ball_snap_fid + 20 and game_play_id==@gpid'),
    gpid, 
    every_other_frame=True, 
    event_col='event_new',
    plot_motion=False,
    show_motion_frames=False,
    highlight_offensive_positions=True,
)